# Loading data with CombinedCesnetDataset

### Import

In [1]:
from tqdm import tqdm
import logging

from cesnet_tszoo.utils.enums import AgreggationType, SourceType, TimeFormat, DatasetType
from cesnet_tszoo.datasets import CESNET_TimeSeries24
from cesnet_tszoo.configs import DisjointTimeBasedConfig # Disjoint dataset MUST use DisjointTimeBasedConfig

### Setting logger

In [2]:
logging.basicConfig(
    level=logging.INFO,
    format="[%(asctime)s][%(name)s][%(levelname)s] - %(message)s")

### Preparing dataset

In [3]:
disjoint_dataset = CESNET_TimeSeries24.get_dataset(data_root="/some_directory/", source_type=SourceType.INSTITUTION_SUBNETS, aggregation=AgreggationType.AGG_1_HOUR, dataset_type=DatasetType.DISJOINT_TIME_BASED, display_details=True)

[2025-08-19 10:50:42,266][wrapper_dataset][INFO] - Dataset is disjoint_time_based. Use cesnet_tszoo.configs.DisjointTimeBasedConfig



Dataset details:

    AgreggationType.AGG_1_HOUR
        Time indices: range(0, 6717)
        Datetime: (datetime.datetime(2023, 10, 9, 0, 0, tzinfo=datetime.timezone.utc), datetime.datetime(2024, 7, 14, 21, 0, tzinfo=datetime.timezone.utc))

    SourceType.INSTITUTION_SUBNETS
        Time series indices: [0 1 2 3 4 ... 543 544 545 546 547], Length=548; use 'get_available_ts_indices' for full list
        Features with default values: {'n_flows': 0, 'n_packets': 0, 'n_bytes': 0, 'tcp_udp_ratio_packets': 0.5, 'tcp_udp_ratio_bytes': 0.5, 'dir_ratio_packets': 0.5, 'dir_ratio_bytes': 0.5, 'avg_duration': 0, 'avg_ttl': 0, 'sum_n_dest_asn': 0, 'avg_n_dest_asn': 0, 'std_n_dest_asn': 0, 'sum_n_dest_ports': 0, 'avg_n_dest_ports': 0, 'std_n_dest_ports': 0, 'sum_n_dest_ip': 0, 'avg_n_dest_ip': 0, 'std_n_dest_ip': 0}
        
        Additional data: ['ids_relationship', 'weekends_and_holidays']
        


### Loading data with DataLoader

- Load data using Pytorch Dataloader.
- Last batch is never dropped (unless sliding_window is used).
- Workers affect how many processes will be used for loading data for specific set.
    - Workers set to 0, means loading will be ran on main process.
    - Set workers can be overriden in `get_*_dataloader` with parameter `workers`.
- Batch size affect how many times for every time series will be in one batch (differs when sliding window is used).
- Batch consists of: (only when sliding window is not used).
    - When `time_format` is not TimeFormat.DATETIME, then batch is one Numpy array of shape `(train_ts/val_ts/test_ts, batch_size, features_to_take + used ids)`.
    - When `time_format` is TimeFormat.DATETIME, then batch is a tuple: (Numpy array of shape `(train_ts/val_ts/test_ts, batch_size, features_to_take + used ids (without time))`, Numpy array of shape `(batch_size)`)

In [4]:
config = DisjointTimeBasedConfig(train_ts=0.5, val_ts=0.2, test_ts=0.1, train_time_period=0.5, val_time_period=0.3, test_time_period=0.2, features_to_take="all", time_format=TimeFormat.ID_TIME,
                         train_workers=0, val_workers=0, test_workers=0, init_workers=0,
                         train_batch_size=32, val_batch_size=64, test_batch_size=128)
disjoint_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-08-19 10:50:42,271][combined_config][INFO] - Quick validation succeeded.
[2025-08-19 10:50:42,292][combined_config][INFO] - Finalization and validation completed successfully.
[2025-08-19 10:50:42,296][cesnet_dataset][INFO] - Updating config for train set.
100%|██████████| 274/274 [00:00<00:00, 1315.64it/s]
[2025-08-19 10:50:42,523][cesnet_dataset][INFO] - Updating config for val set.
100%|██████████| 109/109 [00:00<00:00, 1021.78it/s]
[2025-08-19 10:50:42,643][cesnet_dataset][INFO] - Updating config for test set.
100%|██████████| 54/54 [00:00<00:00, 1066.87it/s]
[2025-08-19 10:50:42,700][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_1_HOUR
    Source: SourceType.INSTITUTION_SUBNETS

    Time series
        Train time series IDs: [132 204 455 456  18 ...  40 130 305 504 182], Length=274
        Val time series IDs: [221  27 151 271 268 ... 491  76  98 334 331], Length=109
        Test time series IDs: [540 286 297 411 487 ... 289 472 226 395 311], Length=54
    Time periods
        Train time periods: range(0, 3359)
        Val time periods: range(3359, 5374)
        Test time periods: range(5374, 6717)
    Features
        Taken features: ['n_flows', 'n_packets', 'n_bytes', 'sum_n_dest_asn', 'avg_n_dest_asn', 'std_n_dest_asn', 'sum_n_dest_ports', 'avg_n_dest_ports', 'std_n_dest_ports', 'sum_n_dest_ip', 'avg_n_dest_ip', 'std_n_dest_ip', 'tcp_udp_ratio_packets', 'tcp_udp_ratio_bytes', 'dir_ratio_packets', 'dir_ratio_bytes', 'avg_duration', 'avg_ttl']
        Default values: [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 

You can also change set batch sizes later with `update_dataset_config_and_initialize` or `set_batch_sizes`.

In [5]:
disjoint_dataset.update_dataset_config_and_initialize(train_batch_size=33, val_batch_size=65, test_batch_size="config")
# Or
disjoint_dataset.set_batch_sizes(train_batch_size=33, val_batch_size=65, test_batch_size="config")

[2025-08-19 10:50:42,706][cesnet_dataset][INFO] - Re-initialization is not needed.
[2025-08-19 10:50:42,706][cesnet_dataset][INFO] - Configuration has been changed successfuly.
[2025-08-19 10:50:42,707][cesnet_dataset][INFO] - Re-initialization is not needed.
[2025-08-19 10:50:42,707][cesnet_dataset][INFO] - Configuration has been changed successfuly.
[2025-08-19 10:50:42,707][cesnet_dataset][INFO] - Batch sizes has been changed successfuly.


You can also change set workers later with `update_dataset_config_and_initialize` or `set_workers`.

In [6]:
disjoint_dataset.update_dataset_config_and_initialize(train_workers=0, val_workers=0, test_workers=0, init_workers=0)
# Or
disjoint_dataset.set_workers(train_workers=0, val_workers=0, test_workers=0, init_workers=0)

[2025-08-19 10:50:42,712][cesnet_dataset][INFO] - Re-initialization is not needed.
[2025-08-19 10:50:42,712][cesnet_dataset][INFO] - Configuration has been changed successfuly.
[2025-08-19 10:50:42,713][cesnet_dataset][INFO] - Re-initialization is not needed.
[2025-08-19 10:50:42,714][cesnet_dataset][INFO] - Configuration has been changed successfuly.
[2025-08-19 10:50:42,714][cesnet_dataset][INFO] - Workers has been changed successfuly.


#### Train set

- Affected by `train_batch_size`.
- Affected by `train_workers`.

In [7]:
dataloader = disjoint_dataset.get_train_dataloader(workers="config")

batches = []

for batch in tqdm(dataloader):
    batches.append(batch)
    
display(batches[0].shape)

[2025-08-19 10:50:42,722][cesnet_dataset][INFO] - Created new cached train_dataloader.
100%|██████████| 105/105 [00:02<00:00, 46.33it/s]


(274, 32, 20)

#### Val set

- Affected by `val_batch_size`.
- Affected by `val_workers`.

In [8]:
dataloader = disjoint_dataset.get_val_dataloader(workers="config")

batches = []

for batch in tqdm(dataloader):
    batches.append(batch)
    
display(batches[0].shape)

[2025-08-19 10:50:45,001][cesnet_dataset][INFO] - Created new cached val_dataloader.
100%|██████████| 32/32 [00:00<00:00, 65.32it/s]


(109, 64, 20)

#### Test set

- Affected by `test_batch_size`.
- Affected by `test_workers`.

In [9]:
dataloader = disjoint_dataset.get_test_dataloader(workers="config")

batches = []

for batch in tqdm(dataloader):
    batches.append(batch)
    
display(batches[0].shape)

[2025-08-19 10:50:45,505][cesnet_dataset][INFO] - Created new cached test_dataloader.
100%|██████████| 11/11 [00:00<00:00, 98.86it/s]


(54, 128, 20)

#### Using time_format=TimeFormat.DATETIME

In [10]:
config = DisjointTimeBasedConfig(train_ts=0.5, val_ts=0.2, test_ts=0.1, train_time_period=0.5, val_time_period=0.3, test_time_period=0.2, features_to_take="all", time_format=TimeFormat.DATETIME,
                         train_workers=0, val_workers=0, test_workers=0, init_workers=0,
                         train_batch_size=32, val_batch_size=64, test_batch_size=128)
disjoint_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-08-19 10:50:45,625][combined_config][INFO] - Quick validation succeeded.
[2025-08-19 10:50:45,648][combined_config][INFO] - Finalization and validation completed successfully.
[2025-08-19 10:50:45,652][cesnet_dataset][INFO] - Updating config for train set.
100%|██████████| 274/274 [00:00<00:00, 2584.65it/s]
[2025-08-19 10:50:45,763][cesnet_dataset][INFO] - Updating config for val set.
100%|██████████| 109/109 [00:00<00:00, 3258.40it/s]
[2025-08-19 10:50:45,800][cesnet_dataset][INFO] - Updating config for test set.
100%|██████████| 54/54 [00:00<00:00, 3573.51it/s]
[2025-08-19 10:50:45,818][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_1_HOUR
    Source: SourceType.INSTITUTION_SUBNETS

    Time series
        Train time series IDs: [288  37 293  60 422 ... 121  56 232  25 514], Length=274
        Val time series IDs: [142 449 429 510 392 ... 486 315 285  70  16], Length=109
        Test time series IDs: [371 276 226 536 495 ... 340 512 189 186 494], Length=54
    Time periods
        Train time periods: range(0, 3359)
        Val time periods: range(3359, 5374)
        Test time periods: range(5374, 6717)
    Features
        Taken features: ['n_flows', 'n_packets', 'n_bytes', 'sum_n_dest_asn', 'avg_n_dest_asn', 'std_n_dest_asn', 'sum_n_dest_ports', 'avg_n_dest_ports', 'std_n_dest_ports', 'sum_n_dest_ip', 'avg_n_dest_ip', 'std_n_dest_ip', 'tcp_udp_ratio_packets', 'tcp_udp_ratio_bytes', 'dir_ratio_packets', 'dir_ratio_bytes', 'avg_duration', 'avg_ttl']
        Default values: [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 

In [11]:
dataloader = disjoint_dataset.get_train_dataloader(workers="config")

batches = []

for batch in tqdm(dataloader):
    batches.append(batch)
    
display(batches[0][0].shape) # data without time
display(batches[0][1].shape) # time

[2025-08-19 10:50:45,825][cesnet_dataset][INFO] - Created new cached train_dataloader.
100%|██████████| 105/105 [00:02<00:00, 50.06it/s]


(274, 32, 19)

(32,)

#### Specifying which time series to load

- Every `get_*_dataloader` has parameter `ts_id`.
    - When `ts_id` is None, then it returns as previous examples.
    - When `ts_id` is not None, then it returns only one time series of that specified id.

In [12]:
config = DisjointTimeBasedConfig(train_ts=[177, 176, 319, 267], val_ts=None, test_ts=None, train_time_period=0.5, features_to_take="all", time_format=TimeFormat.ID_TIME,
                         train_workers=0, val_workers=0, test_workers=0, init_workers=0,
                         train_batch_size=32, val_batch_size=64, test_batch_size=128)
disjoint_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-08-19 10:50:47,931][combined_config][INFO] - Quick validation succeeded.
[2025-08-19 10:50:47,941][combined_config][INFO] - Finalization and validation completed successfully.
[2025-08-19 10:50:47,945][cesnet_dataset][INFO] - Updating config for train set.
100%|██████████| 4/4 [00:00<00:00, 2001.10it/s]
[2025-08-19 10:50:47,949][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_1_HOUR
    Source: SourceType.INSTITUTION_SUBNETS

    Time series
        Train time series IDs: [177 176 319 267], Length=4
        Val time series IDs: None
        Test time series IDs: None
    Time periods
        Train time periods: range(0, 3359)
        Val time periods: None
        Test time periods: None
    Features
        Taken features: ['n_flows', 'n_packets', 'n_bytes', 'sum_n_dest_asn', 'avg_n_dest_asn', 'std_n_dest_asn', 'sum_n_dest_ports', 'avg_n_dest_ports', 'std_n_dest_ports', 'sum_n_dest_ip', 'avg_n_dest_ip', 'std_n_dest_ip', 'tcp_udp_ratio_packets', 'tcp_udp_ratio_bytes', 'dir_ratio_packets', 'dir_ratio_bytes', 'avg_duration', 'avg_ttl']
        Default values: [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.5 0.5 0.5 0.5 0.  0. ]
        Time series ID included: True
        Time included: True    
        Time format: TimeFormat.ID_TIME
    Sliding window
      

In [13]:
dataloader = disjoint_dataset.get_train_dataloader(ts_id=177 , workers="config",)

batches = []

for batch in tqdm(dataloader):
    batches.append(batch)
    
display(batches[0].shape)

[2025-08-19 10:50:47,958][cesnet_dataset][INFO] - Created new cached train_dataloader.
100%|██████████| 105/105 [00:00<00:00, 1691.21it/s]


(1, 32, 20)

#### Sliding window

- Both `sliding_window_size` and `sliding_window_prediction_size` must be set if you want to use sliding window.
- Batch sizes are used for background caching.
- Batch consists of:
    - When `time_format` is not TimeFormat.DATETIME, then batch is a tuple: <br>
    ( <br>
        Numpy array of shape `(train_ts/val_ts/test_ts, sliding_window_size, features_to_take + used ids)`, <br>
        Numpy array of shape `(train_ts/val_ts/test_ts, sliding_window_prediction_size, features_to_take + used ids)` <br>
    ).
    - When `time_format` is TimeFormat.DATETIME, then batch is a tuple: <br>
    ( <br>
        Numpy array of shape `(train_ts/val_ts/test_ts, sliding_window_size, features_to_take + used ids (without time))`, <br>
        Numpy array of shape `(train_ts/val_ts/test_ts, sliding_window_prediction_size, features_to_take + used ids (without time))`, <br>
        Numpy array of shape `(sliding_window_size)` of time, <br>
        Numpy array of shape `(sliding_window_prediction_size)` of time, <br>
    ).
- You can modify sliding window step size with `sliding_window_step`
- You can use `set_shared_size` to set how many times time periods should share.
    - `val_time_period` takes from `train_time_period`
    - `test_time_period` takes from `val_time_period` or `train_time_period`

In [14]:
config = DisjointTimeBasedConfig(train_ts=0.5, val_ts=0.2, test_ts=0.1, train_time_period=range(0, 1000), val_time_period=range(1000, 1500), test_time_period=range(1500, 2000), features_to_take=["n_flows"], time_format=TimeFormat.ID_TIME,
                         train_workers=0, val_workers=0, test_workers=0, init_workers=0,
                         train_batch_size=32, val_batch_size=64, test_batch_size=128,
                         sliding_window_size=22, sliding_window_prediction_size=2, sliding_window_step=2, set_shared_size=0.05)
disjoint_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-08-19 10:50:48,032][combined_config][INFO] - Quick validation succeeded.
[2025-08-19 10:50:48,052][combined_config][INFO] - Finalization and validation completed successfully.
[2025-08-19 10:50:48,055][cesnet_dataset][INFO] - Updating config for train set.
100%|██████████| 274/274 [00:00<00:00, 7702.42it/s]
[2025-08-19 10:50:48,097][cesnet_dataset][INFO] - Updating config for val set.
100%|██████████| 109/109 [00:00<00:00, 6114.72it/s]
[2025-08-19 10:50:48,121][cesnet_dataset][INFO] - Updating config for test set.
100%|██████████| 54/54 [00:00<00:00, 5978.73it/s]
[2025-08-19 10:50:48,132][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_1_HOUR
    Source: SourceType.INSTITUTION_SUBNETS

    Time series
        Train time series IDs: [240 208 277 475 111 ...  45 320 449 350 276], Length=274
        Val time series IDs: [ 25 102 487 182 265 ... 408 395 520 322 382], Length=109
        Test time series IDs: [178 220   9 241 397 ... 521  28 353 197 250], Length=54
    Time periods
        Train time periods: range(0, 1000)
        Val time periods: range(665, 1500)
        Test time periods: range(1165, 2000)
    Features
        Taken features: ['n_flows']
        Default values: [0.]
        Time series ID included: True
        Time included: True    
        Time format: TimeFormat.ID_TIME
    Sliding window
        Sliding window size: 22
        Sliding window prediction size: 2
        Sliding window step size: 2
    Fillers
        Filler type: None
    Transformers
        Transformer type: None
    Batch sizes
       

In [15]:
dataloader = disjoint_dataset.get_train_dataloader(workers="config")

batches = []

for sliding_window, sliding_window_prediction in tqdm(dataloader):
    batches.append((sliding_window, sliding_window_prediction))

[2025-08-19 10:50:48,140][cesnet_dataset][INFO] - Created new cached train_dataloader.
100%|██████████| 489/489 [00:00<00:00, 1086.51it/s]


You can also change sliding window parameters later with `update_dataset_config_and_initialize` or `set_sliding_window`.

In [16]:
disjoint_dataset.update_dataset_config_and_initialize(sliding_window_size=22, sliding_window_prediction_size=3, sliding_window_step="config", set_shared_size="config", workers=0)
# Or
disjoint_dataset.set_sliding_window(sliding_window_size=22, sliding_window_prediction_size=3, sliding_window_step="config", set_shared_size="config", workers=0)

[2025-08-19 10:50:48,597][cesnet_dataset][INFO] - Re-initialization is not needed.
[2025-08-19 10:50:48,597][combined_config][INFO] - all_batch_size adjusted to 25 as it should be greater than or equal to sliding_window_size + sliding_window_prediction_size.
[2025-08-19 10:50:48,598][cesnet_dataset][INFO] - Destroyed cached train_dataloader.
[2025-08-19 10:50:48,598][cesnet_dataset][INFO] - Configuration has been changed successfuly.
[2025-08-19 10:50:48,599][cesnet_dataset][INFO] - Re-initialization is not needed.
[2025-08-19 10:50:48,599][cesnet_dataset][INFO] - Configuration has been changed successfuly.
[2025-08-19 10:50:48,600][cesnet_dataset][INFO] - Sliding window values has been changed successfuly.


##### Using time_format=TimeFormat.DATETIME

In [17]:
config = DisjointTimeBasedConfig(train_ts=0.5, val_ts=0.2, test_ts=0.1, train_time_period=range(0, 1000), val_time_period=range(978, 1500), test_time_period=range(1478, 2000), features_to_take=["n_flows"], time_format=TimeFormat.DATETIME,
                         train_workers=0, val_workers=0, test_workers=0, init_workers=0,
                         train_batch_size=32, val_batch_size=64, test_batch_size=128,
                         sliding_window_size=22, sliding_window_prediction_size=2, sliding_window_step=2, set_shared_size=100)
disjoint_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-08-19 10:50:48,605][combined_config][INFO] - Quick validation succeeded.
[2025-08-19 10:50:48,625][combined_config][INFO] - Finalization and validation completed successfully.
[2025-08-19 10:50:48,628][cesnet_dataset][INFO] - Updating config for train set.
100%|██████████| 274/274 [00:00<00:00, 8207.74it/s]
[2025-08-19 10:50:48,666][cesnet_dataset][INFO] - Updating config for val set.
100%|██████████| 109/109 [00:00<00:00, 8714.15it/s]
[2025-08-19 10:50:48,685][cesnet_dataset][INFO] - Updating config for test set.
100%|██████████| 54/54 [00:00<00:00, 6373.07it/s]
[2025-08-19 10:50:48,695][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_1_HOUR
    Source: SourceType.INSTITUTION_SUBNETS

    Time series
        Train time series IDs: [278 336 524 401 520 ...  72  48 129 108 547], Length=274
        Val time series IDs: [ 52 349 305 183  86 ... 239 267 269 280  42], Length=109
        Test time series IDs: [ 78 321 140 383 209 ... 135  97  66 409 528], Length=54
    Time periods
        Train time periods: range(0, 1000)
        Val time periods: range(900, 1500)
        Test time periods: range(1400, 2000)
    Features
        Taken features: ['n_flows']
        Default values: [0.]
        Time series ID included: True
        Time included: True    
        Time format: TimeFormat.DATETIME
    Sliding window
        Sliding window size: 22
        Sliding window prediction size: 2
        Sliding window step size: 2
    Fillers
        Filler type: None
    Transformers
        Transformer type: None
    Batch sizes
      

In [18]:
dataloader = disjoint_dataset.get_train_dataloader(workers="config")

batches = []

for sliding_window, sliding_window_prediction, sliding_window_times, sliding_window_prediction_times in tqdm(dataloader):
    batches.append((sliding_window, sliding_window_prediction, sliding_window_times, sliding_window_prediction_times))

[2025-08-19 10:50:48,705][cesnet_dataset][INFO] - Created new cached train_dataloader.
100%|██████████| 489/489 [00:00<00:00, 1228.23it/s]


### Loading data as Dataframe

- Batch size has no effect.
- Sliding window has no effect.
- Returns every time series in `train_ts/val_ts/test_ts` with sets specified time period.
- Data is returned as Pandas Dataframe.
- Workers affect how many processes will be used for loading data for specific set.
    - Workers set to 0, means loading will be ran on main process.
    - Set workers can be overriden in `get_*_df` with parameter `workers`.

In [19]:
config = DisjointTimeBasedConfig(train_ts=0.5, val_ts=0.2, test_ts=0.1, train_time_period=0.5, val_time_period=0.3, test_time_period=0.2, features_to_take="all", time_format=TimeFormat.ID_TIME,
                         train_workers=0, val_workers=0, test_workers=0, init_workers=0)
disjoint_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-08-19 10:50:49,109][combined_config][INFO] - Quick validation succeeded.
[2025-08-19 10:50:49,129][combined_config][INFO] - Finalization and validation completed successfully.
[2025-08-19 10:50:49,132][cesnet_dataset][INFO] - Updating config for train set.
100%|██████████| 274/274 [00:00<00:00, 2780.02it/s]
[2025-08-19 10:50:49,235][cesnet_dataset][INFO] - Updating config for val set.
100%|██████████| 109/109 [00:00<00:00, 3378.93it/s]
[2025-08-19 10:50:49,272][cesnet_dataset][INFO] - Updating config for test set.
100%|██████████| 54/54 [00:00<00:00, 3696.93it/s]
[2025-08-19 10:50:49,288][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_1_HOUR
    Source: SourceType.INSTITUTION_SUBNETS

    Time series
        Train time series IDs: [388 179 180 488 269 ... 411  15  11 501 443], Length=274
        Val time series IDs: [ 34 154  25 121 234 ... 365 362 508  98 297], Length=109
        Test time series IDs: [368 512 197  65 399 ... 287 321 441  71  32], Length=54
    Time periods
        Train time periods: range(0, 3359)
        Val time periods: range(3359, 5374)
        Test time periods: range(5374, 6717)
    Features
        Taken features: ['n_flows', 'n_packets', 'n_bytes', 'sum_n_dest_asn', 'avg_n_dest_asn', 'std_n_dest_asn', 'sum_n_dest_ports', 'avg_n_dest_ports', 'std_n_dest_ports', 'sum_n_dest_ip', 'avg_n_dest_ip', 'std_n_dest_ip', 'tcp_udp_ratio_packets', 'tcp_udp_ratio_bytes', 'dir_ratio_packets', 'dir_ratio_bytes', 'avg_duration', 'avg_ttl']
        Default values: [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 

#### Train set

- Affected by `train_workers`.

In [20]:
df = disjoint_dataset.get_train_df(as_single_dataframe=True, workers="config")
dfs = disjoint_dataset.get_train_df(as_single_dataframe=False, workers="config")

df.head(10)

,id_institution_subnet,id_time,n_flows,n_packets,n_bytes,sum_n_dest_asn,avg_n_dest_asn,std_n_dest_asn,sum_n_dest_ports,avg_n_dest_ports,std_n_dest_ports,sum_n_dest_ip,avg_n_dest_ip,std_n_dest_ip,tcp_udp_ratio_packets,tcp_udp_ratio_bytes,dir_ratio_packets,dir_ratio_bytes,avg_duration,avg_ttl
0,388.0,0.0,679.0,17699.0,9137883.0,143.0,11.920000,7.330000,101.0,8.42,3.73,300.0,25.000000,20.490000,0.979980,0.939941,0.409912,0.239990,9.730000,160.339996
1,388.0,1.0,620.0,14105.0,6761003.0,139.0,11.580000,7.250000,97.0,8.08,3.06,296.0,24.670000,22.469999,0.939941,0.910156,0.370117,0.229980,12.730000,160.449997
2,388.0,2.0,1312.0,544225.0,738767768.0,221.0,22.100000,18.820000,92.0,9.20,5.22,737.0,73.699997,74.239998,0.790039,0.779785,0.300049,0.199951,16.920000,165.389999
3,388.0,3.0,2463.0,219134.0,229962357.0,293.0,24.420000,21.600000,140.0,11.67,7.49,1387.0,115.580002,126.320000,0.919922,0.919922,0.360107,0.239990,13.310000,144.690002
4,388.0,4.0,3593.0,422291.0,452458531.0,316.0,26.330000,24.910000,124.0,10.33,7.70,1804.0,150.330002,158.320007,0.870117,0.839844,0.439941,0.340088,11.820000,149.009995
5,388.0,5.0,2898.0,288039.0,259821202.0,334.0,33.400002,25.290001,129.0,12.90,7.50,1664.0,166.399994,142.759995,0.810059,0.759766,0.429932,0.280029,16.280001,140.190002
6,388.0,6.0,3445.0,368034.0,383137472.0,315.0,28.639999,25.490000,116.0,10.55,7.50,1834.0,166.729996,165.550003,0.819824,0.839844,0.310059,0.209961,12.850000,125.320000
7,388.0,7.0,2597.0,167910.0,139096245.0,302.0,25.170000,23.969999,124.0,10.33,7.45,1481.0,123.419998,134.139999,0.859863,0.810059,0.379883,0.260010,14.520000,171.639999
8,388.0,8.0,2920.0,421457.0,463314176.0,334.0,27.830000,25.879999,110.0,9.17,5.73,1684.0,140.330002,144.339996,0.770020,0.700195,0.310059,0.209961,13.290000,131.940002
9,388.0,9.0,1562.0,193100.0,89362048.0,256.0,23.270000,23.080000,136.0,12.36,10.04,942.0,85.639999,103.779999,0.939941,0.910156,0.419922,0.250000,14.750000,156.580002


In [21]:
dfs

[      id_institution_subnet  id_time  n_flows  n_packets      n_bytes  \
 0                     388.0      0.0    679.0    17699.0    9137883.0   
 1                     388.0      1.0    620.0    14105.0    6761003.0   
 2                     388.0      2.0   1312.0   544225.0  738767768.0   
 3                     388.0      3.0   2463.0   219134.0  229962357.0   
 4                     388.0      4.0   3593.0   422291.0  452458531.0   
 ...                     ...      ...      ...        ...          ...   
 3354                  388.0   3354.0    441.0     6910.0    3216701.0   
 3355                  388.0   3355.0    456.0    16850.0    7977410.0   
 3356                  388.0   3356.0    355.0     6163.0    2189509.0   
 3357                  388.0   3357.0    438.0     6047.0    2192821.0   
 3358                  388.0   3358.0    440.0     6428.0    2619324.0   
 
       sum_n_dest_asn  avg_n_dest_asn  std_n_dest_asn  sum_n_dest_ports  \
 0              143.0           11.

#### Val set

- Affected by `val_workers`.

In [22]:
df = disjoint_dataset.get_val_df(as_single_dataframe=True, workers="config")
dfs = disjoint_dataset.get_val_df(as_single_dataframe=False, workers="config")

df.head(10)

,id_institution_subnet,id_time,n_flows,n_packets,n_bytes,sum_n_dest_asn,avg_n_dest_asn,std_n_dest_asn,sum_n_dest_ports,avg_n_dest_ports,std_n_dest_ports,sum_n_dest_ip,avg_n_dest_ip,std_n_dest_ip,tcp_udp_ratio_packets,tcp_udp_ratio_bytes,dir_ratio_packets,dir_ratio_bytes,avg_duration,avg_ttl
0,34.0,3359.0,85.0,1419.0,117936.0,45.0,3.75,1.54,34.0,2.83,1.19,73.0,6.08,3.42,0.409912,0.399902,0.469971,0.469971,90.580002,178.000000
1,34.0,3360.0,109.0,2351.0,197308.0,49.0,3.50,1.61,40.0,2.86,1.66,91.0,6.50,4.20,0.290039,0.250000,0.479980,0.479980,85.769997,151.630005
2,34.0,3361.0,78.0,1525.0,126821.0,40.0,3.08,1.32,37.0,2.85,1.14,74.0,5.69,3.22,0.419922,0.399902,0.569824,0.580078,82.669998,184.830002
3,34.0,3362.0,75.0,1564.0,130477.0,46.0,3.07,1.75,37.0,2.47,1.25,69.0,4.60,3.14,0.340088,0.310059,0.549805,0.529785,76.180000,150.580002
4,34.0,3363.0,89.0,1500.0,124668.0,54.0,3.38,1.31,42.0,2.62,1.54,79.0,4.94,2.32,0.209961,0.189941,0.449951,0.439941,78.129997,119.389999
5,34.0,3364.0,48.0,1014.0,84304.0,31.0,2.21,0.70,27.0,1.93,0.83,45.0,3.21,1.42,0.429932,0.429932,0.589844,0.560059,78.889999,168.539993
6,34.0,3365.0,59.0,1269.0,105589.0,35.0,2.50,0.94,28.0,2.00,0.88,54.0,3.86,2.07,0.330078,0.300049,0.500000,0.509766,89.300003,171.419998
7,34.0,3366.0,26.0,313.0,25831.0,19.0,2.71,1.25,16.0,2.29,1.38,25.0,3.57,1.40,0.429932,0.429932,0.580078,0.580078,86.739998,190.470001
8,34.0,3367.0,64.0,751.0,61593.0,44.0,3.14,1.51,38.0,2.71,1.44,59.0,4.21,2.33,0.439941,0.419922,0.549805,0.549805,67.309998,173.600006
9,34.0,3368.0,56.0,1015.0,83875.0,37.0,2.64,1.45,34.0,2.43,1.28,54.0,3.86,2.51,0.399902,0.399902,0.520020,0.520020,78.940002,164.220001


In [23]:
dfs

[      id_institution_subnet  id_time  n_flows  n_packets   n_bytes  \
 0                      34.0   3359.0     85.0     1419.0  117936.0   
 1                      34.0   3360.0    109.0     2351.0  197308.0   
 2                      34.0   3361.0     78.0     1525.0  126821.0   
 3                      34.0   3362.0     75.0     1564.0  130477.0   
 4                      34.0   3363.0     89.0     1500.0  124668.0   
 ...                     ...      ...      ...        ...       ...   
 2010                   34.0   5369.0     99.0     1331.0  110634.0   
 2011                   34.0   5370.0     89.0     1422.0  118194.0   
 2012                   34.0   5371.0    118.0     1599.0  131665.0   
 2013                   34.0   5372.0     85.0     1390.0  115695.0   
 2014                   34.0   5373.0     82.0     1722.0  143839.0   
 
       sum_n_dest_asn  avg_n_dest_asn  std_n_dest_asn  sum_n_dest_ports  \
 0               45.0            3.75            1.54              34.0

#### Test set

- Affected by `test_workers`.

In [24]:
df = disjoint_dataset.get_test_df(as_single_dataframe=True, workers="config")
dfs = disjoint_dataset.get_test_df(as_single_dataframe=False, workers="config")

df.head(10)

,id_institution_subnet,id_time,n_flows,n_packets,n_bytes,sum_n_dest_asn,avg_n_dest_asn,std_n_dest_asn,sum_n_dest_ports,avg_n_dest_ports,std_n_dest_ports,sum_n_dest_ip,avg_n_dest_ip,std_n_dest_ip,tcp_udp_ratio_packets,tcp_udp_ratio_bytes,dir_ratio_packets,dir_ratio_bytes,avg_duration,avg_ttl
0,368.0,5374.0,1458.0,17303.0,6.953577e+06,148.0,13.450000,11.860000,85.0,7.730000,4.880000,443.0,40.270000,35.799999,0.879883,0.870117,0.449951,0.330078,3.30,134.429993
1,368.0,5375.0,1412.0,17138.0,5.987169e+06,110.0,10.000000,7.750000,82.0,7.450000,5.280000,396.0,36.000000,32.200001,0.910156,0.930176,0.569824,0.449951,5.52,160.750000
2,368.0,5376.0,1371.0,17381.0,5.405039e+06,111.0,12.330000,8.050000,88.0,9.780000,5.520000,384.0,42.669998,30.180000,0.979980,0.990234,0.500000,0.310059,3.86,131.539993
3,368.0,5377.0,1793.0,38166.0,2.649268e+07,195.0,21.670000,15.570000,108.0,12.000000,7.430000,690.0,76.669998,60.619999,0.979980,0.990234,0.500000,0.320068,5.78,144.160004
4,368.0,5378.0,1972.0,55882.0,4.275733e+07,230.0,20.910000,18.209999,128.0,11.640000,9.530000,793.0,72.089996,69.879997,0.830078,0.839844,0.520020,0.379883,4.47,166.720001
5,368.0,5379.0,7560.0,511275.0,5.212858e+08,396.0,36.000000,35.950001,239.0,21.730000,19.620001,2744.0,249.449997,315.079987,0.850098,0.839844,0.399902,0.300049,6.22,139.520004
6,368.0,5380.0,18508.0,1538049.0,1.512350e+09,621.0,88.709999,38.459999,256.0,36.570000,16.360001,5949.0,849.859985,386.390015,0.890137,0.890137,0.350098,0.290039,12.84,112.139999
7,368.0,5381.0,15613.0,2731280.0,3.162274e+09,610.0,55.450001,51.650002,260.0,23.639999,20.040001,5474.0,497.640015,483.929993,0.819824,0.790039,0.320068,0.239990,9.04,133.740005
8,368.0,5382.0,19500.0,2558341.0,2.639545e+09,690.0,62.730000,59.400002,211.0,19.180000,16.100000,6570.0,597.270020,582.400024,0.899902,0.879883,0.409912,0.320068,10.28,145.240005
9,368.0,5383.0,18765.0,1726645.0,1.714331e+09,683.0,75.889999,54.810001,187.0,20.780001,13.390000,6866.0,762.890015,571.500000,0.910156,0.870117,0.399902,0.290039,12.96,124.430000


In [25]:
dfs

[      id_institution_subnet  id_time  n_flows  n_packets     n_bytes  \
 0                     368.0   5374.0   1458.0    17303.0   6953577.0   
 1                     368.0   5375.0   1412.0    17138.0   5987169.0   
 2                     368.0   5376.0   1371.0    17381.0   5405039.0   
 3                     368.0   5377.0   1793.0    38166.0  26492675.0   
 4                     368.0   5378.0   1972.0    55882.0  42757327.0   
 ...                     ...      ...      ...        ...         ...   
 1338                  368.0   6712.0    474.0    10215.0   4552780.0   
 1339                  368.0   6713.0    521.0    10897.0   7666760.0   
 1340                  368.0   6714.0    598.0    27013.0  27657173.0   
 1341                  368.0   6715.0    462.0    10806.0   6417463.0   
 1342                  368.0   6716.0    374.0     8065.0   3370929.0   
 
       sum_n_dest_asn  avg_n_dest_asn  std_n_dest_asn  sum_n_dest_ports  \
 0              148.0           13.45       11.

### Loading data as singular Numpy array 

- Batch size has no effect.
- Sliding window has no effect.
- Returns every time series in `train_ts/val_ts/test_ts` with sets specified time period.
- Data is returned as one Numpy array.
- Follows similar rules to Dataloader batches, regarding shape (excluding sliding window parameters).
- Workers affect how many processes will be used for loading data for specific set.
    - Workers set to 0, means loading will be ran on main process.
    - Set workers can be overriden in `get_*_numpy` with parameter `workers`.

In [26]:
config = DisjointTimeBasedConfig(train_ts=0.5, val_ts=0.2, test_ts=0.1, train_time_period=0.5, val_time_period=0.3, test_time_period=0.2, features_to_take="all", time_format=TimeFormat.ID_TIME,
                         train_workers=0, val_workers=0, test_workers=0, init_workers=0)
disjoint_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-08-19 10:50:51,818][combined_config][INFO] - Quick validation succeeded.
[2025-08-19 10:50:51,839][combined_config][INFO] - Finalization and validation completed successfully.
[2025-08-19 10:50:51,843][cesnet_dataset][INFO] - Updating config for train set.
100%|██████████| 274/274 [00:00<00:00, 2711.96it/s]
[2025-08-19 10:50:51,949][cesnet_dataset][INFO] - Updating config for val set.
100%|██████████| 109/109 [00:00<00:00, 3196.97it/s]
[2025-08-19 10:50:51,988][cesnet_dataset][INFO] - Updating config for test set.
100%|██████████| 54/54 [00:00<00:00, 3414.06it/s]
[2025-08-19 10:50:52,006][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_1_HOUR
    Source: SourceType.INSTITUTION_SUBNETS

    Time series
        Train time series IDs: [112  51  19  13 452 ... 172 455 425 504 219], Length=274
        Val time series IDs: [142 407 314  44 381 ... 216 522 345 151 467], Length=109
        Test time series IDs: [262 222 237 468 276 ... 304 434  95  90 526], Length=54
    Time periods
        Train time periods: range(0, 3359)
        Val time periods: range(3359, 5374)
        Test time periods: range(5374, 6717)
    Features
        Taken features: ['n_flows', 'n_packets', 'n_bytes', 'sum_n_dest_asn', 'avg_n_dest_asn', 'std_n_dest_asn', 'sum_n_dest_ports', 'avg_n_dest_ports', 'std_n_dest_ports', 'sum_n_dest_ip', 'avg_n_dest_ip', 'std_n_dest_ip', 'tcp_udp_ratio_packets', 'tcp_udp_ratio_bytes', 'dir_ratio_packets', 'dir_ratio_bytes', 'avg_duration', 'avg_ttl']
        Default values: [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 

#### Train set

- Affected by `train_workers`.

In [27]:
numpy_array = disjoint_dataset.get_train_numpy(workers="config")

display(numpy_array.shape)

(274, 3359, 20)

#### Val set

- Affected by `val_workers`.

In [28]:
numpy_array = disjoint_dataset.get_val_numpy(workers="config")

display(numpy_array.shape)

(109, 2015, 20)

#### Test set

- Affected by `test_workers`.

In [29]:
numpy_array = disjoint_dataset.get_test_numpy(workers="config")

display(numpy_array.shape)

(54, 1343, 20)

#### Using time_format=TimeFormat.DATETIME

In [30]:
config = DisjointTimeBasedConfig(train_ts=0.5, val_ts=0.2, test_ts=0.1, train_time_period=0.5, val_time_period=0.3, test_time_period=0.2, features_to_take="all", time_format=TimeFormat.DATETIME,
                         train_workers=0, val_workers=0, test_workers=0, init_workers=0)
disjoint_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-08-19 10:50:52,388][combined_config][INFO] - Quick validation succeeded.
[2025-08-19 10:50:52,411][combined_config][INFO] - Finalization and validation completed successfully.
[2025-08-19 10:50:52,414][cesnet_dataset][INFO] - Updating config for train set.
100%|██████████| 274/274 [00:00<00:00, 2805.03it/s]
[2025-08-19 10:50:52,518][cesnet_dataset][INFO] - Updating config for val set.
100%|██████████| 109/109 [00:00<00:00, 3025.75it/s]
[2025-08-19 10:50:52,558][cesnet_dataset][INFO] - Updating config for test set.
100%|██████████| 54/54 [00:00<00:00, 3371.38it/s]
[2025-08-19 10:50:52,576][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_1_HOUR
    Source: SourceType.INSTITUTION_SUBNETS

    Time series
        Train time series IDs: [443 273  37 228 270 ... 190 237 159 421 143], Length=274
        Val time series IDs: [154 243 233  68 399 ...  13 296  49  60 413], Length=109
        Test time series IDs: [195 442  90 423 130 ...  31 346 463 387 420], Length=54
    Time periods
        Train time periods: range(0, 3359)
        Val time periods: range(3359, 5374)
        Test time periods: range(5374, 6717)
    Features
        Taken features: ['n_flows', 'n_packets', 'n_bytes', 'sum_n_dest_asn', 'avg_n_dest_asn', 'std_n_dest_asn', 'sum_n_dest_ports', 'avg_n_dest_ports', 'std_n_dest_ports', 'sum_n_dest_ip', 'avg_n_dest_ip', 'std_n_dest_ip', 'tcp_udp_ratio_packets', 'tcp_udp_ratio_bytes', 'dir_ratio_packets', 'dir_ratio_bytes', 'avg_duration', 'avg_ttl']
        Default values: [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 

In [31]:
numpy_array, times = disjoint_dataset.get_train_numpy(workers="config")

display(numpy_array.shape)
display(times)

(274, 3359, 19)

array([datetime.datetime(2023, 10, 9, 0, 0, tzinfo=datetime.timezone.utc),
       datetime.datetime(2023, 10, 9, 1, 0, tzinfo=datetime.timezone.utc),
       datetime.datetime(2023, 10, 9, 2, 0, tzinfo=datetime.timezone.utc),
       ...,
       datetime.datetime(2024, 2, 25, 20, 0, tzinfo=datetime.timezone.utc),
       datetime.datetime(2024, 2, 25, 21, 0, tzinfo=datetime.timezone.utc),
       datetime.datetime(2024, 2, 25, 22, 0, tzinfo=datetime.timezone.utc)],
      shape=(3359,), dtype=object)